In [ ]:
import argparse
import datetime
import http.client
import json
import logging
import math
import multiprocessing as mp
import numpy as np
import os
import pandas as pd
import re
from datatypes.panorama import Panorama
import requests
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
base_url = "https://sidewalk-amsterdam.cs.washington.edu/v2/access/attributesWithLabels?lat1={}&lng1={}&lat2={}&lng2={}" 
whole = (52.303, 4.8, 52.425, 5.05)
centrum_west = (52.364925, 4.87444, 52.388692, 4.90641)
test = (52.0, 4.0, 53.0, 5.0)

coords = test

url = base_url.format(*coords)

local_dump = url.replace('/', '|')

try:
    project_sidewalk_labels = json.load(open(local_dump, 'r'))
except Exception as e:
    print("Couldn't load local dump")
    project_sidewalk_labels = requests.get(url.format(*coords)).json()
    json.dump(project_sidewalk_labels, open(local_dump, 'w'))

In [ ]:
ps_labels_df = gpd.GeoDataFrame.from_features(project_sidewalk_labels['features'])
# Print length before filtering
print('Length before filtering:', len(ps_labels_df))

In [ ]:
# Send a get call to this API: https://sidewalk-amsterdam-test.cs.washington.edu/adminapi/labels/cvMetadata
other_labels = requests.get('https://sidewalk-amsterdam.cs.washington.edu/adminapi/labels/cvMetadata').json()
other_labels_df = pd.DataFrame(other_labels)
print(len(other_labels_df))

In [ ]:
# Filter labels dataframe to only contain obstacles
ps_labels_df = ps_labels_df[ps_labels_df['label_type'] == 'Obstacle']

# Intersect other_labels_df with ps_labels_df
other_labels_df = other_labels_df[other_labels_df['gsv_panorama_id'].isin(ps_labels_df['gsv_panorama_id'])]
print('Number of labels in other_labels_df after filtering for obstacles: ', len(other_labels_df))
print(other_labels_df[['gsv_panorama_id','sv_image_x', 'sv_image_y']].head(20))

In [ ]:
def visualize_label(dataframe, pano_id):

    image_path = f'../sidewalk-accessibility-features/res/dataset_PS/centrum_west_small/reoriented/{pano_id}'
    mask_path = f'../sidewalk-accessibility-features/res/dataset_PS/centrum_west_small/backprojected/{pano_id}/{pano_id}.png'
    image = plt.imread(image_path)
    mask = plt.imread(mask_path)
    print(f'The shape of the image is: {image.shape}')

    # Find all the labels in the panorama
    labels = dataframe[dataframe['gsv_panorama_id'] == pano_id]

    labels_coords = []

    # For each label, find the canvas_x, canvas_y, canvas_width, canvas_height
    for index, row in labels.iterrows():
        print('Labels coming from panorama:', row['gsv_panorama_id'])
        label_name = row['label_id']
        image_width = row['image_width']
        image_height = row['image_height']
        sv_image_x = row['sv_image_x']
        # Momentarily change the definition of sv_image_y. For now,
        # sv_image_y is the offset w.r.t. the middle of the image
        # which means that the real y coordinate is y = (image_width / 2) - sv_image_y
        sv_image_y = row['sv_image_y']
        real_y = (image_height / 2) - sv_image_y

        print(f'Original image width: {image_width}, height: {image_height}')
        print(f'Original label {label_name} coordinates: {sv_image_x}, {sv_image_y}')
        print(f'Original label {label_name} with real y-coordinates' \
            f'({image_height}/2){sv_image_y}: {sv_image_x}, {real_y}')

        # image_width : sv_image_x = my_pano_width : sv_pano_x
        # sv_pano_x = (sv_image_x * my_pano_width) / image_width
        sv_pano_x = int(sv_image_x*image.shape[1]/image_width)
        sv_pano_y = int(real_y*image.shape[0]/image_height)

        print(f'Scaled label {label_name} coordinates: {sv_pano_x}, {sv_pano_y} \n')

        # Visualize the point (sv_pano_x, sv_pano_y) on the image
        plt.imshow(image, cmap='gray')
        plt.imshow(mask, cmap='jet', alpha=0.4)
        plt.scatter(sv_pano_x, sv_pano_y, color='red', s=10)

        labels_coords.append((sv_pano_y, sv_pano_x))
    # Make a folder 'visualized' if not present
    if not os.path.exists(f'../sidewalk-accessibility-features/res/dataset_PS/centrum_west_small/visualized'):
        os.makedirs(f'../sidewalk-accessibility-features/res/dataset_PS/centrum_west_small/visualized')
    # Save the image
    plt.savefig(f'../sidewalk-accessibility-features/res/dataset_PS/centrum_west_small/visualized/{pano_id}.png', dpi=300, bbox_inches='tight')
        
    return labels_coords

In [ ]:
def split_masks(pred_mask):
    # Determine the center of the image
    center = pred_mask.shape[1] // 2

    # Create two new images with the same dimensions as the original image
    mask1 = np.zeros_like(pred_mask)
    mask2 = np.zeros_like(pred_mask)

    # Copy the first half of the original image to the first new image
    # and blacken the second half
    mask1[:, :center] = pred_mask[:, :center]

    # Copy the second half of the original image to the second new image
    # and blacken the first half
    mask2[:, center:] = pred_mask[:, center:]

    return mask1, mask2

In [ ]:
def compute_distances(gt_points, pred_masks):
    distances = []
    closest_points = []
    mask_indices = []

    for gt_point in gt_points:
        min_distance = float('inf')
        closest_y, closest_x = -1, -1
        mask_index = -1

        for idx, pred_mask in enumerate(pred_masks):
            mask_channel = pred_mask[:, :, 0]  # Assuming the first channel contains the binary mask
            mask_coords = np.where(mask_channel > 0)  # Get the indices of non-zero elements (i.e., the mask)
            mask_coords = np.vstack(mask_coords).T  # Stack the mask indices into a 2D array

            point_coords = np.array(gt_point).reshape(1, -1)
            dist = cdist(point_coords, mask_coords)

            local_min_distance_idx = np.argmin(dist)
            local_min_distance = dist[0, local_min_distance_idx]
            local_closest_y, local_closest_x = mask_coords[local_min_distance_idx]

            if local_min_distance < min_distance:
                min_distance = local_min_distance
                closest_y, closest_x = local_closest_y, local_closest_x
                mask_index = idx

        distances.append(min_distance)
        closest_points.append((closest_y, closest_x))
        mask_indices.append(mask_index)

    return distances, closest_points, mask_indices

def visualize_debug(gt_points, pred_masks, distances, closest_points, mask_indices):
    num_masks = len(pred_masks)
    fig, axes = plt.subplots(num_masks, 1, figsize=(8, 8 * num_masks))

    for idx, (pred_mask, ax) in enumerate(zip(pred_masks, axes)):
        ax.imshow(pred_mask)  # Display the predicted mask

        for gt_point, distance, closest_point, mask_index in zip(gt_points, distances, closest_points, mask_indices):
            if mask_index == idx:
                y, x = gt_point
                closest_y, closest_x = closest_point

                print(f"Ground truth point: {gt_point}, Closest point: {closest_point}, Distance: {distance}")

                ax.scatter(x, y, c='red', marker='x', s=50, label='Ground Truth')  # Mark the ground truth point
                ax.plot([x, closest_x], [y, closest_y], 'r--', label='Distance')  # Draw the distance line
                ax.scatter(closest_x, closest_y, c='blue', marker='o', s=20, label='Closest Point')  # Mark the closest point

        ax.set_title(f"Closest point-to-mask distance for mask {idx + 1}")
        ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Choose a random pano_id from other_labels_df
pano_id = other_labels_df['gsv_panorama_id'].sample(1).values[0]
#pano_id = '4mn-4MZ5Pw9MbgfdR_LOMg'
print(f'Pano ID: {pano_id}')
mask_path = f'../sidewalk-accessibility-features/res/dataset_PS/centrum_west_small/backprojected/{pano_id}/{pano_id}.png'
pred_masks = plt.imread(mask_path)
print(f'The shape of the mask is: {pred_masks.shape}')
mask1, mask2 = split_masks(pred_masks)

In [ ]:
# Print sv_image_x and sv_image_y of label_id 3822 in other_labels_df
#print('Before changing values: ' , other_labels_df.loc[other_labels_df['label_id'] == 3822, ['sv_image_x', 'sv_image_y']])

# Change sv_image_x and sv_image_y of label_id 3822 in other_labels_df to (5203, -1023)
#other_labels_df.loc[other_labels_df['label_id'] == 3822, 'sv_image_x'] = 5203
#other_labels_df.loc[other_labels_df['label_id'] == 3822, 'sv_image_y'] = -1023

gt_points = visualize_label(other_labels_df, pano_id)
pred_masks = [mask1, mask2]

#print(gt_points)
# Change the first gt_points pair values to (5203,-1023)
#gt_points[0] = (5203, -1023)

distances, closest_points, mask_indices = compute_distances(gt_points, pred_masks)
visualize_debug(gt_points, pred_masks, distances, closest_points, mask_indices)